<a href="https://colab.research.google.com/github/DiegoPincheiraIb/Experiencias_EL5206/blob/master/Proyecto_Imagenes_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final Area Imagenes - Luckas Diaz y Diego Pincheira

El objetivo de este proyecto final es desarrollar e implementar un algoritmo de búsqueda de imágenes similares basado en su contenido (Content Based Image Retrieval, CBIR). Para el desarrollo de este proyecto debe descargar la base de datos INRIA Holidays dataset. Debe descargar los archivos jpg1.tar.gz y jpg2.tar.gz.

Link de para descargar: http://lear.inrialpes.fr/people/jegou/data.php

Las imágenes de la base de datos sólo se pueden utilizar en este proyecto académico. Para cualquier otro uso se debe referenciar el trabajo: Herve Jegou, Matthijs Douze and Cordelia Schmid, "Hamming Embedding and Weak geometry consistency for large scale image search", Proceedings of the 10th European conference on Computer vision, October, 2008.

# Imports

Se importan las librerías a utilizar.

In [ ]:
import os

import matplotlib.pyplot as plt
import matplotlib.image as mpimg 

import skimage.io as io
from skimage import color, io, exposure
from skimage.feature import hog

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances
import numpy as np
from IPython.display import clear_output
import cv2
from google.colab.patches import cv2_imshow
from scipy import ndimage, misc
from natsort import natsorted
from scipy.spatial import distance
import random
!pip install ipython-autotime
%load_ext autotime

# Cargar datos a Colab

## Desde disco local

Aqui se deben cargar los archivos jpg1.tar.gz y jpg2.tar.gz

In [ ]:
from google.colab import files
uploaded = files.upload()

## Desde Drive
Se requiere aqui que los archivos jpg1.tar.gz y jpg2.tar.gz se encuentren en una carpeta llamada "Proyecto-Imagenes" dentro del directorio Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls

In [ ]:
%cd drive/MyDrive/Proyecto-Imagenes/

In [ ]:
!ls
# En pantalla debería mostrarse: jpg1.tar.gz  jpg2.tar.gz

In [ ]:
!tar -xvf  'jpg1.tar.gz'
!tar -xvf  'jpg2.tar.gz'
clear_output()
print('listo!')

In [ ]:
!ls
# En pantalla debe mostrarse: jpg  jpg1.tar.gz  jpg2.tar.gz

## Directo desde la página

In [ ]:
!wget ftp://ftp.inrialpes.fr/pub/lear/douze/data/jpg1.tar.gz
!wget ftp://ftp.inrialpes.fr/pub/lear/douze/data/jpg2.tar.gz

In [ ]:
!ls
# En pantalla debería mostrarse: jpg1.tar.gz  jpg2.tar.gz

In [ ]:

!tar -xvf  'jpg1.tar.gz'
!tar -xvf  'jpg2.tar.gz'
clear_output()
print('listo!')

In [ ]:
!ls
# En pantalla debe mostrarse: jpg  jpg1.tar.gz  jpg2.tar.gz

# Obtener paths de imagenes

Se obtiene una lista con los nombres de las imágenes a utilizar.

In [ ]:
path_imagenes = 'jpg/'
Listado_imagenes = natsorted(os.listdir(path_imagenes))

# Carga de Imágenes

Se elabora la función "Carga_Imgs_Resize", la cual, dado un directorio objetivo y un listado de los nombres de las imágenes a cargar, genera una lista con la información de las imagenes objetivo, con un tamaño de 256x256.

In [ ]:
def Carga_Imgs_Resize(Path_Obj,Listado_Obj, Factor_Reescala_Obj = 1):
  Lista_Completa = []
  for imagen in Listado_Obj:
    Lista_Completa.append(cv2.resize(cv2.imread(Path_Obj + '/' + imagen),(256,256)))
  return Lista_Completa

## Database con imágenes sin procesar

Mediante la función anterior, se crea una lista que contiene las imágenes deseadas.

In [ ]:
Database_Imgs_Raw = Carga_Imgs_Resize(path_imagenes, Listado_imagenes, 0.5)

# Cálculo de características

Se utilizarán dos métodos para obtener características de las imágenes: un descriptor HoG, y VGG16, la cual es una CNN. Se crea la función "Calculo_Vector_Caract()" la cual, dado una imagen objetivo y un string 'hog' o 'cnn' (que determina si se utilizará HoG o CNN), elabora un vector de características de dicha imagen.

Se procede a elaborar el modelo de la CNN:

In [ ]:
model = VGG16(weights='imagenet', include_top=False, pooling = "avg")

Se crea la función definida anteriormente

In [ ]:
def Calculo_Vector_Caract(Img_Obj, Tipo_Vector):
  if Tipo_Vector == 'hog':
    Feature_Vector_Obj = hog(Img_Obj, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(4, 4), feature_vector =True)
  else:
    Img_Obj = cv2.resize(Img_Obj, (224,224))
    Img_Obj = np.expand_dims(image.img_to_array(Img_Obj), axis=0)
    Img_Obj = preprocess_input(Img_Obj)
    Feature_Vector_Obj = model.predict(Img_Obj).flatten()
  return Feature_Vector_Obj

Por último, se elabora la función "Tupla_Caract()", la cual, dado un listado de nombres de imágenes, una Database de imágenes sin procesar, y un string 'hog' o 'cnn', retorna una lista con tuplas de la forma (Caract, Clase, ID), donde el primer elemento consiste en un vector de características elaborado con la función "Calculo_Vector_Caract()", el segundo elemento consiste en la clase de la imagen, y el tercero consiste en la ID de la imagen.

In [ ]:
def Tupla_Caract(Listado_Obj,Database_Obj,Tipo_Vector):
  Lista_Caract_Obj = []
  for i in range((len(Listado_Obj))):
      Lista_Caract_Obj.append((Calculo_Vector_Caract(Database_Obj[i], Tipo_Vector),Listado_Obj[i][1:-6],Listado_Obj[i][-6:-4]))
  return Lista_Caract_Obj

## Distancias:

Se elabora la función "Funcion_Distancia" la que, dado dos vectores y un string que determina el tipo de distancia a utilizar ('eucl', 'chi', 'cos'), retorna la distancia entre ambos vectores utilizando una función determinada por el string entregado

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances

In [ ]:
# Str_Dist = 'eucl', 'cos'

def Funcion_Distancia(Vect_1, Vect_2, Str_Dist = 'eucl'):
  if Str_Dist == 'eucl':
    Distancia = distance.euclidean(Vect_1,Vect_2)
  elif Str_Dist == 'cos':
    Distancia = distance.cosine(Vect_1, Vect_2)
  return Distancia

### Busqueda

Se crea la función "Busqueda" la que, dado un vector de características objetivo, una base de datos de vectores de características objetivo, y un string que determina el tipo de distancia a utilizar ('eucl', 'chi', 'cos'), retorna una lista con las distancia entre el vector objetivo y los elementos de la base de datos.

In [ ]:
# Str_Dist = 'eucl', 'chi', 'cos'
def Busqueda(Feature_Vector_Obj, Database_Obj, Str_Dist = 'eucl'):
  Similitudes = []
  for imagen, j, k in Database_Obj:
    Distancia = Funcion_Distancia(Feature_Vector_Obj, imagen, Str_Dist)
    Similitudes.append(Distancia)
  return Similitudes

# Rankings:

Se elaboran dos funciones: la primera (Average_Ranking()) entrega el promedio de los rankings, mientras que la segunda (Norm_Ranking()) entrega un ranking normalizado

## Average_Ranking

In [ ]:
def Average_Ranking(Feature_Vector_Obj,Nombre_Img, Database_Obj, Str_Dist = 'eucl'):
  Clase = Nombre_Img[1:-6]
  Sim_Array = Busqueda(Feature_Vector_Obj,Database_Obj, Str_Dist)
  Indices = np.argsort(np.argsort(Sim_Array))
  Tupla_Indices = tuple(zip(Indices,[lista[1] for lista in Database_Obj],[lista[2] for lista in Database_Obj]))
  Localizados = [x for (x,y,z) in Tupla_Indices if y == Clase and z !='00']
  print('Ranking de los elementos de la misma clase:',Localizados)
  Average_Rk = np.mean(np.array(Localizados).astype(np.float))
  return Average_Rk


## Norm_Ranking

In [ ]:
def Norm_Ranking(Feature_Vector_Obj,Nombre_Img, Database_Obj, Str_Dist = 'eucl'):
  Clase = Nombre_Img[1:-6]
  Sim_Array = Busqueda(Feature_Vector_Obj,Database_Obj, Str_Dist)
  Indices = np.argsort(np.argsort(Sim_Array))
  Tupla_Indices = tuple(zip(Indices,[lista[1] for lista in Database_Obj],[lista[2] for lista in Database_Obj]))
  Localizados = [x for (x,y,z) in Tupla_Indices if y == Clase and z !='00']
  N_Rel = len(Localizados)
  Average_Rk = np.mean(np.array(Localizados).astype(np.float))
  Num = Average_Rk - (N_Rel*(N_Rel + 1)/2)
  Den = N_Rel * len(Database_Obj)
  return Num/Den

# Imágenes de consulta

In [ ]:
def Consulta(Feature_Vector_Obj,Nombre_Img,Database_Features_Obj,Str_Dist):
  Clase = Nombre_Img[1:-6]
  Sim_Array = Busqueda(Feature_Vector_Obj,Database_Features_Obj, Str_Dist)
  Indices_Resultados = np.argsort(Sim_Array)
  print('Imagen de consulta:', Nombre_Img)
  print('Clase:',Nombre_Img[1:-6])
  cv2_imshow(cv2.resize(cv2.imread(path_imagenes + '/' + Nombre_Img),(256,256)))
  print('-----------------------------------------------')
  print('Imágenes relevantes:')
  for i in Indices_Resultados[1:11]:
    print('Imagen: ', Listado_imagenes[i])
    print('Clase:',Listado_imagenes[i][1:-6],'ID:',Listado_imagenes[i][-6:-4])
    cv2_imshow(Database_Imgs_Raw[int(i)])


# Resultados Preliminares

## Bases de datos:

In [ ]:
Lista_Hog = Tupla_Caract(Listado_imagenes, Database_Imgs_Raw, 'hog')

In [ ]:
Lista_CNN = Tupla_Caract(Listado_imagenes, Database_Imgs_Raw, 'cnn')

## Índices de imágenes de consulta

In [ ]:
Lista_Indices_Imgs_Consulta = []
for i in range((len(Listado_imagenes))):
  if Listado_imagenes[i][-6:-4] == '00':
    Lista_Indices_Imgs_Consulta.append(i)

## Búsqueda utilizando HoG:

### Distancia Euclidiana

In [ ]:
import random
Index =  random.choice(Lista_Indices_Imgs_Consulta)
Img_Obj = Database_Imgs_Raw[Index]
Vector_Caract_Hog = Calculo_Vector_Caract(Img_Obj,'hog')
print('Average Ranking:', Average_Ranking(Vector_Caract_Hog,Listado_imagenes[Index],Lista_Hog,'eucl'))
Consulta(Vector_Caract_Hog,Listado_imagenes[Index],Lista_Hog,'eucl')

### Distancia coseno

In [ ]:
import random
Index =  random.choice(Lista_Indices_Imgs_Consulta)
Img_Obj = Database_Imgs_Raw[Index]
Vector_Caract_Hog = Calculo_Vector_Caract(Img_Obj,'hog')
print('Average Ranking:', Average_Ranking(Vector_Caract_Hog,Listado_imagenes[Index],Lista_Hog,'cos'))
Consulta(Vector_Caract_Hog,Listado_imagenes[Index],Lista_Hog,'cos')

## Búsqueda utilizando CNN

### Distancia euclidiana

In [ ]:
import random
Index =  random.choice(Lista_Indices_Imgs_Consulta)
#Index = 109
Img_Obj = Database_Imgs_Raw[Index]
Vector_Caract_CNN = Calculo_Vector_Caract(Img_Obj,'cnn')
print('Average Ranking:', Average_Ranking(Vector_Caract_CNN,Listado_imagenes[Index],Lista_CNN,'eucl'))
Consulta(Vector_Caract_CNN,Listado_imagenes[Index],Lista_CNN,'eucl')

### Distancia coseno

In [ ]:
import random
Index =  random.choice(Lista_Indices_Imgs_Consulta)
#Index = 109
Img_Obj = Database_Imgs_Raw[Index]
Vector_Caract_CNN = Calculo_Vector_Caract(Img_Obj,'cnn')
print('Average Ranking:', Average_Ranking(Vector_Caract_CNN,Listado_imagenes[Index],Lista_CNN,'cos'))
Consulta(Vector_Caract_CNN,Listado_imagenes[Index],Lista_CNN,'cos')

## Histogramas:

In [ ]:
import matplotlib.pyplot as plt
Rank_Normz_HoG_Eucl = []
for indice in Lista_Indices_Imgs_Consulta:
  Img_Obj = Database_Imgs_Raw[indice]
  Vector_Caract_Hog = Calculo_Vector_Caract(Img_Obj,'hog')
  Rank_Normz_HoG_Eucl.append(Norm_Ranking(Vector_Caract_Hog,Listado_imagenes[indice],Lista_Hog,'eucl'))



# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(Rank_Normz_HoG_Eucl,
                            alpha=0.7, rwidth=0.85)
plt.title('Resultados obtenidos utilizando distancia euclidiana y descriptores HoG')
plt.show()

Rank_Normz_HoG_Cos = []
for indice in Lista_Indices_Imgs_Consulta:
  Img_Obj = Database_Imgs_Raw[indice]
  Vector_Caract_Hog = Calculo_Vector_Caract(Img_Obj,'hog')
  Rank_Normz_HoG_Cos.append(Norm_Ranking(Vector_Caract_Hog,Listado_imagenes[indice],Lista_Hog,'cos'))



# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(Rank_Normz_HoG_Cos,
                            alpha=0.7, rwidth=0.85)
plt.title('Resultados obtenidos utilizando distancia coseno y descriptores HoG')
plt.show()

In [ ]:
Rank_Normz_VGG_Eucl = []
for indice in Lista_Indices_Imgs_Consulta:
  Img_Obj = Database_Imgs_Raw[indice]
  Vector_Caract_VGG = Calculo_Vector_Caract(Img_Obj,'cnn')
  Rank_Normz_VGG_Eucl.append(Norm_Ranking(Vector_Caract_VGG,Listado_imagenes[indice],Lista_CNN,'eucl'))



# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(Rank_Normz_VGG_Eucl,
                            alpha=0.7, rwidth=0.85)
plt.title('Resultados obtenidos utilizando distancia euclidiana y VGG')
plt.show()

Rank_Normz_VGG_Cos = []
for indice in Lista_Indices_Imgs_Consulta:
  Img_Obj = Database_Imgs_Raw[indice]
  Vector_Caract_VGG = Calculo_Vector_Caract(Img_Obj,'cnn')
  Rank_Normz_VGG_Cos.append(Norm_Ranking(Vector_Caract_VGG,Listado_imagenes[indice],Lista_CNN,'cos'))



# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(Rank_Normz_VGG_Cos,
                            alpha=0.7, rwidth=0.85)
plt.title('Resultados obtenidos utilizando distancia coseno y VGG')
plt.xlabel('Valor del ranking normalizado')
plt.ylabel('Cantidad de imágenes con determinado valor de Ranking Normalizado')
plt.show()

In [ ]:
x = [np.mean(Rank_Normz_HoG_Eucl),np.mean(Rank_Normz_HoG_Cos),np.mean(Rank_Normz_VGG_Eucl),np.mean(Rank_Normz_VGG_Cos)]
fig, ax = plt.subplots()
y = ['Hog + Eucl','HoG + Cos','VGG + Eucl','VGG + Cos']
ind = np.arange(len(y))
ax.bar(ind,y)
width = 0.75 # the width of the bars 
plt.title('Resultados obtenidos utilizando distancia coseno y descriptores HoG')
for i, v in enumerate(x):
    ax.text(v + 3, i + .25, str(y), color='blue', fontweight='bold')
ax.set_yticks(ind+width/2)
ax.set_yticklabels(x, minor=False)
plt.title('title')
plt.xlabel('x')
plt.ylabel('y')  
plt.show()

In [ ]:
Promedio = [np.mean(Rank_Normz_HoG_Eucl),np.mean(Rank_Normz_HoG_Cos),np.mean(Rank_Normz_VGG_Eucl),np.mean(Rank_Normz_VGG_Cos)]

fig = plt.bar(['Hog + Eucl','HoG + Cos','VGG + Eucl','VGG + Cos'],Promedio)
plt.title('Resultados obtenidos utilizando distancia coseno y descriptores HoG')
plt.ylabel('Ranking Normalizado')
for value in fig:
    height = value.get_height()
    plt.text(value.get_x() + value.get_width()/2.,
             1.002*height,'%f' % int(height), ha='center', va='bottom')
plt.show()

# Mejoras a modelos (P7)

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential

# check structure and layer names before looping
model.summary()

# loop through layers, add Dropout after layers 'fc1' and 'fc2'
updated_model = Sequential()
for layer in model.layers:
    updated_model.add(layer)
    if layer.name in ['fc1', 'fc2']:
        updated_model.add(Dropout(.2))

model = updated_model

# check structure
model.summary()

In [ ]:
Nueva_Lista_VGG = Lista_CNN.copy()
Nueva_Lista_Hog = Lista_Hog.copy()

## PCA

In [ ]:
def App_PCA(Lista_Obj):
  DF_Og = pd.DataFrame(Lista_Obj[0][0]).T
  for i in range(1,len(Lista_Obj)):
    Aux = pd.DataFrame(Lista_Obj[i][0]).T
    DF_Og = DF_Og.append(Aux, ignore_index=True)
  PCA_Obj = PCA(n_components=100)
  PCA_Data = PCA_Obj.fit_transform(DF_Og)
  DF_Obj = pd.DataFrame(PCA_Data)
  return DF_Obj

In [ ]:
VGG_PCA = App_PCA(Nueva_Lista_VGG)
Hog_PCA = App_PCA(Nueva_Lista_Hog)

## Búsqueda utilizando HoG:

### Distancia Euclidiana

In [ ]:
Index =  random.choice(Lista_Indices_Imgs_Consulta)
Consulta(Hog_PCA.iloc[Index].values,Listado_imagenes[Index],list(zip(pca_data,[x[1] for x in Nueva_Lista_Hog],[x[2] for x in Nueva_Lista_Hog] )),'eucl')

### Distancia coseno

In [ ]:
Index =  random.choice(Lista_Indices_Imgs_Consulta)
Consulta(Hog_PCA.iloc[Index].values,Listado_imagenes[Index],list(zip(pca_data,[x[1] for x in Nueva_Lista_Hog],[x[2] for x in Nueva_Lista_Hog] )),'cos')

## Búsqueda utilizando CNN

### Distancia euclidiana

In [ ]:
Index =  random.choice(Lista_Indices_Imgs_Consulta)
Consulta(VGG_PCA.iloc[Index].values,Listado_imagenes[Index],list(zip(pca_data,[x[1] for x in Nueva_Lista_VGG],[x[2] for x in Nueva_Lista_VGG] )),'eucl')

### Distancia coseno

In [ ]:
Index =  random.choice(Lista_Indices_Imgs_Consulta)
Consulta(VGG_PCA.iloc[Index].values,Listado_imagenes[Index],list(zip(pca_data,[x[1] for x in Nueva_Lista_VGG],[x[2] for x in Nueva_Lista_VGG] )),'cos')

## Clustering

In [ ]:
Lista_Imgs_DB = [x for (x,y,z) in Lista_Hog if z != '00']

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=500, n_init=20, max_iter=20, random_state=20 )
kmeans.fit(Lista_Imgs_DB) # fit retorna a self
y_pred = kmeans.predict(Lista_Imgs_Consulta)
kmeans.labels_

# Obsoleto

In [ ]:
import pandas as pd
data_frame = pd.DataFrame(Nueva_Lista_VGG[0][0]).T
for i in range(1,1491):
  x = pd.DataFrame(Nueva_Lista_VGG[i][0]).T
  data_frame = data_frame.append(x, ignore_index=True)
data_frame.head()

In [ ]:

from sklearn.decomposition import PCA
pca_breast = PCA(n_components=100)
pca_data = pca_breast.fit_transform(data_frame)

In [ ]:

y = pd.DataFrame(pca_data)
y.head()

In [ ]:
Lista_Imgs_Consulta = [x for (x,y,z) in Lista_Hog if z == '00']
print(Lista_Imgs_Consulta)

In [ ]:
Index = 101
img = Database_Imgs_Raw[Index]
Hola = Calculo_Vector_Caract(img,'hog')
print(Average_Ranking(Hola,Listado_imagenes[Index],Lista_Hog,'eucl'))
Consulta(Hola,Listado_imagenes[Index],Lista_Hog,'eucl')

In [ ]:
print(Average_Ranking(Hola,Listado_imagenes[Index],Lista_Hog,'eucl'))
print(Norm_Ranking(Hola,Listado_imagenes[Index],Lista_Hog,'chi'))

In [ ]:
print(Listado_imagenes[Index])

In [ ]:
Consulta(Hola,Listado_imagenes[Index],Lista_Hog,'eucl')

In [ ]:
print(list(enumerate(Listado_jpg1)))

In [ ]:
Sim = Busqueda(Hola,Listado_jpg1[3][1:-6],Lista_Hog)
Indices = np.argsort(np.argsort(Sim))
print(Sim)
print(tuple(zip(Indices,[lista[1] for lista in Lista_Hog],[lista[2] for lista in Lista_Hog])))


In [ ]:
print(Average_Ranking(Hola,Listado_jpg1[3][1:-6],Lista_Hog))

In [ ]:
print([asd[2] for asd in Sim])

In [ ]:
caca = np.argsort(Indices,axis=0)[0:20]
poto = np.reshape(np.array([asd[1] for asd in Sim][0:20]),(-1,1))
pichi = np.concatenate((caca,poto),axis=1)

In [ ]:
qqqq = tuple(map(tuple,pichi))
print([x for x,y,z,k in qqqq if k == '001'])
print(np.mean(np.array([x for x,y,z,k in qqqq if k == '001']).astype(np.float)))

In [ ]:
import scipy.stats as ss
print((ss.rankdata(Sim)-1)[0:16])

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
print(Indices)

In [ ]:
Relevantes = []
N_rel = 1
Rank = np.sum(Relevantes)/N_Rel

In [ ]:
# Str_Dist = 'eucl', 'chi', 'cos'
def Busqueda(Feature_Vector_Obj,Nombre_Img, Database_Obj, Str_Dist = 'eucl'):
  Similitudes = []
  Clase_Obj = Nombre_Img[1:-6]
  for imagen,j, k in Database_Obj:
    Distancia = Funcion_Distancia(Feature_Vector_Obj, imagen,Str_Dist)
    Similitudes.append((Distancia,j,k))
    #print(Distancia,j,k)
  return Similitudes

In [ ]:
def split_list(alist, wanted_parts=1):
    length = len(alist)
    return [ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
             for i in range(wanted_parts) ]

In [ ]:
def Imagenes_Gris_Resize_Dicc(Path_Obj,Listado_Obj, Factor_Reescala_Obj = 1):
  Dicc_Final = {}
  #Lista_1, Lista_2, Lista_3, Lista_4 = split_list(Listado_Obj, wanted_parts=4)
  if Factor_Reescala_Obj < 1:
    for imagen in Listado_Obj:
      Dicc_Final[imagen[1:-6]] = {}
    for imagen in Listado_Obj:
      Dicc_Final[imagen[1:-6]][imagen[-6:-4]] = 
      Lista_Completa.append(cv2.resize(cv2.imread(Path_Obj + '/' + imagen),None,fx=Factor_Reescala_Obj,fy=Factor_Reescala_Obj))
  else:
    for imagen in Listado_Obj:
      Lista_Completa.append(cv2.imread(Path_Obj + '/' + imagen))
  return Lista_Completa

Diccionario con clases y características

In [ ]:
def Dicc_Caract(Listado_Obj,Database_Obj,Tipo_Vector):
  Dicc_Caract_Obj = {}
  for i in range(len(Listado_Obj)):
    Dicc_Caract_Obj[Listado_Obj[i][1:-6]] = {}
  for i in range((len(Listado_Obj))):
    Dicc_Caract_Obj[Listado_Obj[i][1:-6]][Listado_Obj[i][-6:-4]] = Calculo_Vector_Caract(Database_Obj[i], Tipo_Vector)
  return Dicc_Caract_Obj

In [ ]:
print(Listado_jpg1)

In [ ]:
Hola = []
for i in range(20):
  Hola.append((Listado_jpg1[i][1:-6],Listado_jpg1[i][-6:-4]))
print(Hola)

In [ ]:
import random
Dicc_Listado_jpg1 = {}
for i in range(20):
  Key = str(Hola[i][0])
  Dicc_Listado_jpg1[Key] = {}
for i in range(20):
  Key = str(Hola[i][0])
  String = str(Hola[i][1])
  Dicc_Listado_jpg1[Key][String] = random.randint(3, 9)
  
print(Dicc_Listado_jpg1)

In [ ]:
Blank = []
for j in Dicc_Listado_jpg1['003'].values():
  Blank.append(j - 3)
  print(j - 3)
print(np.mean(Blank))

## HOG

In [ ]:
fd, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True, feature_vector =True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

ax1.axis('off')
ax1.imshow(img, cmap=plt.cm.gray)
ax1.set_title('Input image')

# Rescale histogram for better display
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')
plt.show()

VGG

In [ ]:

img_path = path_jpg1 + '/' + Listado_jpg1[1]
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)

In [ ]:
print(len(Listado_jpg1_Gris))

In [ ]:
Listado_jpg1_1, Listado_jpg1_2, Listado_jpg1_3, Listado_jpg1_4 = split_list(Listado_jpg1, wanted_parts=4)
Listado_jpg1_Gris = []
Factor_Reescala = 0.2

In [ ]:
for imagen in Listado_jpg1_1:
  Listado_jpg1_Gris.append(cv2.resize(cv2.imread(path_jpg1 + '/' + imagen,0),None,fx=Factor_Reescala,fy=Factor_Reescala))

In [ ]:
for imagen in Listado_jpg1_2:
  Listado_jpg1_Gris.append(cv2.resize(cv2.imread(path_jpg1 + '/' + imagen,0),None,fx=Factor_Reescala,fy=Factor_Reescala))

In [ ]:
for imagen in Listado_jpg1_3:
  Listado_jpg1_Gris.append(cv2.resize(cv2.imread(path_jpg1 + '/' + imagen,0),None,fx=Factor_Reescala,fy=Factor_Reescala))

In [ ]:
for imagen in Listado_jpg1_4:
  Listado_jpg1_Gris.append(cv2.resize(cv2.imread(path_jpg1 + '/' + imagen,0),None,fx=Factor_Reescala,fy=Factor_Reescala))

### Jpg2

In [ ]:
Listado_jpg2_1, Listado_jpg2_2, Listado_jpg2_3, Listado_jpg2_4 = split_list(Listado_jpg2, wanted_parts=4)

In [ ]:
Listado_jpg2_Gris = []
for imagen in Listado_jpg2_1:
  Listado_jpg2_Gris.append(cv2.imread(path_jpg2 + '/' + imagen,0))

In [ ]:
for imagen in Listado_jpg2_2:
  Listado_jpg2_Gris.append(cv2.imread(path_jpg2 + '/' + imagen,0))

In [ ]:
for imagen in Listado_jpg2_3:
  Listado_jpg2_Gris.append(cv2.imread(path_jpg2 + '/' + imagen,0))

In [ ]:
for imagen in Listado_jpg2_4:
  Listado_jpg2_Gris.append(cv2.imread(path_jpg2 + '/' + imagen,0))